In [1]:
# import library
import pandas as pd
import re

In [2]:
# import data
excel_file_path = 'Data_Transaksi_2023(terbaru).xlsx'
excel_sheets = pd.read_excel(excel_file_path, sheet_name=None)
excel_sheets

{'Januari':       NO.TRANSAKSI                     DESCRIPTION  QTY
 0     2.301011e+12          MADU NSTR SUPER 650ML.    1
 1              NaN  STELLA MTC F/C 225ML.REF   /12    1
 2              NaN              GELAS KOPI BOLA@50    2
 3              NaN                      MIKA BX IV    3
 4     2.301011e+12  LAGIE GOLD CM 75G.W/30 LG07507    2
 ...            ...                             ...  ...
 2240           NaN    RINSO MOLTO ROSE FRESH 1.8KG    1
 2241           NaN             KRESEK 35/0.02  /50    3
 2242           NaN    VANISH 425ML.REFILL /12 220K    1
 2243           NaN         ULTRA MILK PLN 250ML/24    4
 2244           NaN   SARI ROTI TAWAR JUMBO SPECIAL    1
 
 [2245 rows x 3 columns],
 'Februari':       NO.TRANSAKSI                     DESCRIPTION  QTY
 0     2.302011e+12                 CLEO  6LT.GALON    1
 1     2.302011e+12         KUNYIT ASAM JAWA 350 ML    1
 2     2.302011e+12  FRESCO KOPI+GULA@10        /12    2
 3              NaN  LE MINERALE 600M

# **Pre-processing**

### **1. Merging Data**

In [3]:
df_list = [df for df in excel_sheets.values()]
merged_df = pd.concat(df_list, ignore_index=True)
merged_df 

,NO.TRANSAKSI,DESCRIPTION,QTY
0,2.301011e+12,MADU NSTR SUPER 650ML.,1
1,NaN,STELLA MTC F/C 225ML.REF /12,1
2,NaN,GELAS KOPI BOLA@50,2
3,NaN,MIKA BX IV,3
4,2.301011e+12,LAGIE GOLD CM 75G.W/30 LG07507,2
...,...,...,...
21929,NaN,INDOMIE SOTO MIE/40 SM,1
21930,NaN,INDOMIE KARI AYAM/40 IKA,1
21931,NaN,INDOMIE KALDU AYAM/40 KA75,3
21932,NaN,SEDAAP MIE KOREAN SPCY40-20234,1


### **2. Handling Missing Values**

In [4]:
merged_df['NO.TRANSAKSI'] = merged_df['NO.TRANSAKSI'].ffill()
merged_df

,NO.TRANSAKSI,DESCRIPTION,QTY
0,2.301011e+12,MADU NSTR SUPER 650ML.,1
1,2.301011e+12,STELLA MTC F/C 225ML.REF /12,1
2,2.301011e+12,GELAS KOPI BOLA@50,2
3,2.301011e+12,MIKA BX IV,3
4,2.301011e+12,LAGIE GOLD CM 75G.W/30 LG07507,2
...,...,...,...
21929,2.312311e+12,INDOMIE SOTO MIE/40 SM,1
21930,2.312311e+12,INDOMIE KARI AYAM/40 IKA,1
21931,2.312311e+12,INDOMIE KALDU AYAM/40 KA75,3
21932,2.312311e+12,SEDAAP MIE KOREAN SPCY40-20234,1


### **3. Extract Features**

In [5]:
print(merged_df.dtypes)

NO.TRANSAKSI    float64
DESCRIPTION      object
QTY              object
dtype: object


In [6]:
def convert_to_datetime(transaction_no):
    if pd.isna(transaction_no):
        return None
    transaction_no = str(transaction_no)
    date_str = '20' + transaction_no[:6] 
    return pd.to_datetime(date_str, format='%Y%m%d')

merged_df['DATE'] = merged_df['NO.TRANSAKSI'].apply(convert_to_datetime)
merged_df = merged_df[['NO.TRANSAKSI', 'DATE', 'DESCRIPTION', 'QTY']]
merged_df

,NO.TRANSAKSI,DATE,DESCRIPTION,QTY
0,2.301011e+12,2023-01-01,MADU NSTR SUPER 650ML.,1
1,2.301011e+12,2023-01-01,STELLA MTC F/C 225ML.REF /12,1
2,2.301011e+12,2023-01-01,GELAS KOPI BOLA@50,2
3,2.301011e+12,2023-01-01,MIKA BX IV,3
4,2.301011e+12,2023-01-01,LAGIE GOLD CM 75G.W/30 LG07507,2
...,...,...,...,...
21929,2.312311e+12,2023-12-31,INDOMIE SOTO MIE/40 SM,1
21930,2.312311e+12,2023-12-31,INDOMIE KARI AYAM/40 IKA,1
21931,2.312311e+12,2023-12-31,INDOMIE KALDU AYAM/40 KA75,3
21932,2.312311e+12,2023-12-31,SEDAAP MIE KOREAN SPCY40-20234,1


### **4. Pre-processing Text**

In [7]:
def preprocess_description(text):
    # Hapus spasi double dan teks setelahnya
    text = re.sub(r'\s{2,}.*', '', text)
    # Hapus teks setelah tanda '/'
    text = re.sub(r'/.*', '', text)
    # Hapus tanda baca
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

merged_df["DESCRIPTION_CLEANED"] = merged_df["DESCRIPTION"].apply(preprocess_description)
merged_df

C:\Users\ASUS\AppData\Local\Temp\ipykernel_16860\591640966.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_df["DESCRIPTION_CLEANED"] = merged_df["DESCRIPTION"].apply(preprocess_description)


,NO.TRANSAKSI,DATE,DESCRIPTION,QTY,DESCRIPTION_CLEANED
0,2.301011e+12,2023-01-01,MADU NSTR SUPER 650ML.,1,MADU NSTR SUPER 650ML
1,2.301011e+12,2023-01-01,STELLA MTC F/C 225ML.REF /12,1,STELLA MTC F
2,2.301011e+12,2023-01-01,GELAS KOPI BOLA@50,2,GELAS KOPI BOLA50
3,2.301011e+12,2023-01-01,MIKA BX IV,3,MIKA BX IV
4,2.301011e+12,2023-01-01,LAGIE GOLD CM 75G.W/30 LG07507,2,LAGIE GOLD CM 75GW
...,...,...,...,...,...
21929,2.312311e+12,2023-12-31,INDOMIE SOTO MIE/40 SM,1,INDOMIE SOTO MIE
21930,2.312311e+12,2023-12-31,INDOMIE KARI AYAM/40 IKA,1,INDOMIE KARI AYAM
21931,2.312311e+12,2023-12-31,INDOMIE KALDU AYAM/40 KA75,3,INDOMIE KALDU AYAM
21932,2.312311e+12,2023-12-31,SEDAAP MIE KOREAN SPCY40-20234,1,SEDAAP MIE KOREAN SPCY4020234


# **Save Final Data**

In [8]:
final_df = merged_df[['NO.TRANSAKSI', 'DATE', 'DESCRIPTION_CLEANED', 'QTY']]
final_df

,NO.TRANSAKSI,DATE,DESCRIPTION_CLEANED,QTY
0,2.301011e+12,2023-01-01,MADU NSTR SUPER 650ML,1
1,2.301011e+12,2023-01-01,STELLA MTC F,1
2,2.301011e+12,2023-01-01,GELAS KOPI BOLA50,2
3,2.301011e+12,2023-01-01,MIKA BX IV,3
4,2.301011e+12,2023-01-01,LAGIE GOLD CM 75GW,2
...,...,...,...,...
21929,2.312311e+12,2023-12-31,INDOMIE SOTO MIE,1
21930,2.312311e+12,2023-12-31,INDOMIE KARI AYAM,1
21931,2.312311e+12,2023-12-31,INDOMIE KALDU AYAM,3
21932,2.312311e+12,2023-12-31,SEDAAP MIE KOREAN SPCY4020234,1


In [9]:
# final_df.to_excel('final_data.xlsx', index=False)